In [ ]:

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, DateType, DecimalType

In [2]:
spark = SparkSession.builder.appName("Task3").getOrCreate()

In [ ]:
schema = StructType([
    StructField("transaction_unique_identifier", StringType(), True),
    StructField("price", DecimalType(10, 2), True),
    StructField("date_of_transfer", DateType(), True),
    StructField("postcode", StringType(), True),
    StructField("property_type", StringType(), True),
    StructField("old_new", StringType(), True),
    StructField("duration", StringType(), True),
    StructField("paon", StringType(), True),
    StructField("saon", StringType(), True),
    StructField("street", StringType(), True),
    StructField("locality", StringType(), True),
    StructField("town_city", StringType(), True),
    StructField("district", StringType(), True),
    StructField("county", StringType(), True),
    StructField("ppd_category_type", StringType(), True),
    StructField("record_status", StringType(), True)
])

df = (
    spark.read.format("csv")
    .option("header", "true")
    .schema(schema)
    .load("../data/*.csv") 
)
df = df.select(
    "transaction_unique_identifier",
    "date_of_transfer",
    "price",
    "postcode"
)
df.show()

+-----------------------------+----------------+----------+--------+
|transaction_unique_identifier|date_of_transfer|     price|postcode|
+-----------------------------+----------------+----------+--------+
|         {E53EDD2E-C6F1-83...|      2021-11-08| 150000.00|LS27 9AL|
|         {E53EDD2E-C6F3-83...|      2021-10-29| 430000.00| LS6 1BU|
|         {E53EDD2E-C6F4-83...|      2021-10-26| 135000.00|LS10 1LP|
|         {E53EDD2E-C704-83...|      2021-12-21| 131000.00| HD2 2SN|
|         {E53EDD2E-C705-83...|      2021-10-26| 180000.00|LS10 1NG|
|         {E53EDD2E-C707-83...|      2021-12-23|  65000.00| HD4 6DL|
|         {E53EDD2E-C708-83...|      2021-10-26| 180000.00|LS10 1NW|
|         {E53EDD2E-C709-83...|      2021-12-20|  60000.00| BD4 7EJ|
|         {E53EDD2E-C712-83...|      2021-09-29| 255000.00|BD10 0QZ|
|         {E53EDD2E-C716-83...|      2021-07-16|  93500.00|LS12 5LT|
|         {E53EDD2E-C71C-83...|      2021-12-17|6492000.00|WF13 2SU|
|         {E53EDD2E-C71E-83...|   

In [41]:
df = (
    df
    .withColumns({
        "year": F.year(F.col("date_of_transfer")),
        "postcode_area": F.split(F.col("postcode"), " ").getItem(0)
    })

)
df = df.filter((F.col("year") >= 1995) & (F.col("year") <= 2021))


In [ ]:
from pyspark.sql.window import Window
medians = (
    df.groupBy("postcode_area", "year")
    .agg(F.expr("percentile_approx(price, 0.5)").alias("median_price"))
)

w = Window.partitionBy("postcode_area").orderBy("year")

medians = medians.withColumn(
    "prev_median", F.lag("median_price").over(w)
).withColumn(
    "median_diff", F.abs(F.col("median_price") - F.col("prev_median"))
)

medians_filtered = medians.filter(
    (F.col("postcode_area").isNotNull()) & (F.col("median_diff").isNotNull())
)

diff_sum = (
    medians_filtered.groupBy("postcode_area")
    .agg(F.sum("median_diff").alias("cumulative_median_diff_sum"))
    .orderBy("cumulative_median_diff_sum")
    .limit(25)
)

diff_sum.show()

+-------------+--------------------------+
|postcode_area|cumulative_median_diff_sum|
+-------------+--------------------------+
|          L66|                   4000.00|
|          CF7|                   8000.00|
|          RG3|                  10000.00|
|          L43|                  12000.00|
|         GL30|                  28000.00|
|          NP5|                  90000.00|
|          L41|                  94500.00|
|         DN31|                  98874.00|
|          ST1|                 100210.00|
|          ST6|                 104292.00|
|         LA14|                 107100.00|
|          TD5|                 109000.00|
|          SR8|                 109100.00|
|         CF43|                 110395.00|
|          TS1|                 113000.00|
|         DN32|                 114400.00|
|          HU5|                 114720.00|
|          W1R|                 115050.00|
|          HU9|                 115550.00|
|          SR4|                 115900.00|
+----------